In [ ]:
#!/usr/bin/env python
# coding: utf-8
#######################################################################
# How to setup AndroSensor:
# Install AndroSensor
# Ensure when installing, allow app to run in background.
# Open settings and change recording interval to fast.
#
#
#
#######################################################################
import glob
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'
import numpy as np
import scipy as sp
import scipy.signal as sg

In [ ]:
df = pd.read_csv("SDS00002_noisy_pandas.csv")

In [ ]:
df.head(100)

In [ ]:
df = df.iloc[::20000, :] # Keep one in 2000 samples

In [ ]:
fig = px.line(df, x="Second", y="Volt", title='Signal vs time')
fig.show()

In [ ]:
#%matplotlib qt  # wx, gtk, osx, tk, empty uses default
%matplotlib qt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting

fig, ax = plt.subplots()
ax.plot(df["Second"], df["Volt"])



plt.show()

In [ ]:
# importing modules
import numpy
from matplotlib import pyplot as plt
   
# assigning time values of the signal
# initial time period, final time period and phase angle  
signalTime = numpy.arange(1, 100, 0.5);
  
# getting the amplitude of the signal
signalAmplitude = numpy.sin(signalTime)
  
# plotting the signal 
# plt.plot(signalTime, signalAmplitude, color ='green')
  
# plt.xlabel('Time')
# plt.ylabel('Amplitude')
# plt.title("Signal")
  
  
# plotting the magnitude spectrum of the signal 
plt.magnitude_spectrum(df["Volt"],Fs=20000000, color ='green')
  
plt.title("Magnitude Spectrum of the Signal")
plt.show()

In [ ]:
freqs, psd = signal.welch(df["Volt"],fs=20000000)

plt.figure(figsize=(5, 4))
plt.plot(freqs, psd)
plt.title('PSD: power spectral density Welch' )
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.tight_layout()
plt.show()


In [ ]:
# use psd
plt.psd(df["Volt"], Fs=20000000)

plt.show()

### Use butterworth filter

In [ ]:
import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
%matplotlib qt


def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Filter requirements.
order = 4
fs = 10000      # sample rate, Hz
cutoff = 1000  # desired cutoff frequency of the filter, Hz

# Get the filter coefficients so we can check its frequency response.
b, a = butter_lowpass(cutoff, fs, order)

# Plot the frequency response.
w, h = freqz(b, a, worN=8000)
plt.subplot(2, 1, 1)
plt.plot(0.5*fs*w/np.pi, np.abs(h), 'b')
plt.plot(cutoff, 0.5*np.sqrt(2), 'ko')
plt.axvline(cutoff, color='k')
plt.xlim(0, 0.5*fs)
plt.title("Lowpass Filter Frequency Response")
plt.xlabel('Frequency [Hz]')
plt.grid()


# Demonstrate the use of the filter.
# First make some data to be filtered.
#T = 5.0         # seconds
n = len(df["Volt"])#int(T * fs) # total number of samples
t =df["Second"]# np.linspace(0, T, n, endpoint=False)
# "Noisy" data.  We want to recover the 1.2 Hz signal from this.
data = df["Volt"] #= np.sin(1.2*2*np.pi*t) + 1.5*np.cos(9*2*np.pi*t) + 0.5*np.sin(12.0*2*np.pi*t)

# Filter the data, and plot both the original and filtered signals.
y = butter_lowpass_filter(data, cutoff, fs, order)

plt.subplot(2, 1, 2)
plt.plot(t, data, 'b-', label='data')
plt.plot(t, y, 'g-', linewidth=2, label='filtered data')
plt.xlabel('Time [sec]')
plt.grid()
plt.legend()

plt.subplots_adjust(hspace=0.35)
plt.show()

### Use simple threshold filter

In [ ]:
import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
%matplotlib qt

dead_high = 3.5
dead_low = 0.4

def threshold_lowpass_filter(df, dead_low, dead_high):
    
    df = df[~df['Volt'].between(dead_low, dead_high)]
    return df

t = df["Second"]
data = df["Volt"]

# Filter the data, and plot both the original and filtered signals.
data_filtered = threshold_lowpass_filter(df, dead_low, dead_high)

plt.subplot(2, 1, 2)
plt.plot(t, data, 'b-', label='data')
plt.plot(data_filtered["Second"], data_filtered["Volt"], 'g-', linewidth=2, label='filtered data')
plt.xlabel('Time [sec]')
plt.grid()
plt.legend()

plt.show()